In [ ]:
!git clone https://github.com/adnan119/TRACER.git

In [ ]:
!wget https://github.com/Karel911/TRACER/releases/download/v1.0/TRACER-Efficient-7.pth

In [ ]:
!mv ./TRACER-Efficient-7.pth ./best_model.pth

In [ ]:
!pip install timm faiss-gpu

In [ ]:
!pip install fastai --upgrade

In [ ]:
!pip uninstall torch -y
# CUDA 10.1
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install fastai==2.0.19
!pip install fastcore==1.3.1 
!pip install wwf -q --upgrade

In [ ]:
%cd ./TRACER
!mkdir ./results/
!mkdir ./results/DUTS/
!mkdir ./results/DUTS/TE7_0/

In [ ]:
!mv ../best_model.pth ./results/DUTS/TE7_0/

In [ ]:
!pip install -r requirements.txt

In [ ]:
# NB : Restart notebook

In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import gc
from tqdm import tqdm
import fastai
from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer
import faiss
from timm import create_model
from timm.data.transforms_factory import create_transform
from sklearn.preprocessing import normalize

import cv2
import math
import copy
import time
import random
import warnings
import shutil

# For data manipulation
import numpy as np
import pandas as pd
from PIL import *

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# segment all images to remove background
!python main.py test --exp_num 0 --arch 7 --img_size 640 --batch_size 16 --dataset "DUTS" --save_map True --data_path /path/to/images/

In [ ]:
!ls seg_img | wc -c

In [ ]:
shutil.make_archive('./seg_img', 'zip', './seg_img')

In [ ]:
!mv ./TRACER/seg_img.zip .

In [ ]:
!rm -r seg_img
!rm seg_img.zip

In [ ]:
!pwd

In [ ]:
%cd ..

In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [ ]:
!rm -r TRACER

In [ ]:
!unzip seg_img.zip -d clean_turtle_background

In [ ]:
!rm seg_img.zip

In [ ]:
import os

SOURCE_URL = 'https://storage.googleapis.com/dm-turtle-recall/images.tar'
IMAGE_DIR = './turtle_recall/images'
TAR_PATH = os.path.join(IMAGE_DIR, os.path.basename(SOURCE_URL))
EXPECTED_IMAGE_COUNT = 13891

%sx mkdir --parents "{IMAGE_DIR}"
if len(os.listdir(IMAGE_DIR)) != EXPECTED_IMAGE_COUNT:
  %sx wget --no-check-certificate -O "{TAR_PATH}" "{SOURCE_URL}"
  %sx tar --extract --file="{TAR_PATH}" --directory="{IMAGE_DIR}"
  %sx rm "{TAR_PATH}"

print(f'The total number of images is: {len(os.listdir(IMAGE_DIR))}')

In [ ]:
OUTPUT_DIR = Path(".")
DATA_ROOT_DIR = Path("/content/turtle_recall/images")
SUBMISSION_CSV_PATH = OUTPUT_DIR / "submission.csv"

# Loading dataset

In [ ]:
import pandas as pd
import requests
import io
import urllib.parse

BASE_URL = 'https://storage.googleapis.com/dm-turtle-recall/'


def read_csv_from_web(file_name):
  url = urllib.parse.urljoin(BASE_URL, file_name)
  content = requests.get(url).content
  return pd.read_csv(io.StringIO(content.decode('utf-8')))


# Read in csv files.
train = read_csv_from_web('train.csv')
extra_train = read_csv_from_web('extra_images.csv')
test_df = read_csv_from_web('test.csv')
sample_submission = read_csv_from_web('sample_submission.csv')

# Convert image_location strings to lowercase.
for df in [train, test_df]:
  df.image_location = df.image_location.apply(lambda x: x.lower())
  assert set(df.image_location.unique()) == set(['left', 'right', 'top'])

In [ ]:
classes = list(train.turtle_id.unique()) + list(extra_train.turtle_id.unique())

In [ ]:
classes[:5]

In [ ]:
new_data = train.copy()

In [ ]:
new_data['image_id'] = new_data['image_id'] + '.png'
new_data['image_path'] = new_data['image_id'].apply(lambda x : f'/content/clean_turtle_background/{str(x)}')

In [ ]:
extra_1 = extra_train.copy()
extra_1['image_id'] = extra_1['image_id'] + '.JPG'
extra_1['image_path'] = extra_1['image_id'].apply(lambda x : DATA_ROOT_DIR / str(x))

In [ ]:
extra_2 = extra_train.copy()
extra_2['image_id'] = extra_2['image_id'] + '.png'
extra_2['image_path'] = extra_2['image_id'].apply(lambda x : f'/content/clean_turtle_background/{str(x)}')

In [ ]:
extra_train = pd.concat([extra_1, extra_2], 
                        axis=0, ignore_index=True)

In [ ]:
del extra_1, extra_2
gc.collect()

In [ ]:
train['image_id']   = train['image_id'] + '.JPG'
test_df['image_id'] = test_df['image_id'] + '.JPG'

train['image_path']   = train['image_id'].apply(lambda x : DATA_ROOT_DIR / str(x))
test_df['image_path'] = test_df['image_id'].apply(lambda x : DATA_ROOT_DIR / str(x))

# Splitting data

In [ ]:
augmented_train = pd.concat([train, new_data], axis=0, ignore_index=True)
folds = pd.concat([augmented_train, extra_train], axis=0, ignore_index=True)

In [ ]:
N_SPLITS = 5
encoder = LabelEncoder()

encoder.fit(folds['turtle_id'])
np.save('turtle_ids.npy', encoder.classes_)


folds["label"] = encoder.transform(folds['turtle_id'])

skf = StratifiedKFold(n_splits=N_SPLITS, random_state=42, shuffle=True)


for fold, (_, val_) in enumerate(skf.split(X=folds, y=folds.turtle_id)):
    folds.loc[val_, "kfold"] = int(fold)
    
folds.drop('label',axis=1,inplace=True)
folds.head()

In [ ]:
folds['kfold'] = folds['kfold'].astype(int)

In [ ]:
folds

In [ ]:
seed=42
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

# Creating dataloaders

In [ ]:
Val_Fold = 0

def get_x(r): return r['image_path']
def get_y(r): return [r['turtle_id']]

def splitter(df): 

    train_ = df.index[df.kfold != Val_Fold].tolist()

    valid_ = df.index[df.kfold == Val_Fold].tolist()

    return [train_,valid_]

def create_dls(df=folds, bs=8,Val_Fold=0,Image_size=512):
    dblock = DataBlock(blocks = (ImageBlock,MultiCategoryBlock(vocab=classes)),
                       get_x = get_x,
                       get_y = get_y ,
                       splitter = splitter,
                       item_tfms = [Resize(1024)], 
                       batch_tfms =[*aug_transforms(size=Image_size, 
                                                    do_flip=True, flip_vert=True, max_rotate=5.0, 
                                                    p_affine=0.5, p_lighting=0.5, max_zoom=1.08,
                                                    max_lighting=0.2, max_warp=0.2
                                                    ),
                                     Normalize.from_stats(*imagenet_stats)]
                      )

    dls = dblock.dataloaders(folds,bs=bs)

    return dls

In [ ]:
dls = create_dls(df=folds)

In [ ]:
dls.show_batch()

In [ ]:
target_map = {N:CLASS for N,CLASS in enumerate(dls.vocab)}

# Arc Face loss

In [ ]:
# From https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/blob/master/src/modeling/metric_learning.py
# Added type annotations, device, and 16bit support
class ArcMarginLoss(Module):
    def __init__(
        self,
        in_features: int,
        out_features: int,
        s: float,
        m: float,
        easy_margin: bool,
        ls_eps: float,
    ):
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, logits, targ):
        cosine = F.linear(F.normalize(logits), F.normalize(self.weight))
        # Enable 16 bit precision
        cosine = cosine.to(torch.float32)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        if self.ls_eps > 0:
            targ = (1 - self.ls_eps) * targ + self.ls_eps / self.out_features
        output = (targ * phi) + ((1.0 - targ) * cosine)
        output *= self.s
        loss =  F.cross_entropy(output, torch.argmax(targ, dim=1))
        return loss

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, 
                 m=0.50, easy_margin=False, ls_eps=0.0):
        '''
        in_features: dimension of the input
        out_features: dimension of the last layer (in our case the classification)
        s: norm of input feature
        m: margin
        ls_eps: label smoothing'''
        
        super(ArcMarginProduct, self).__init__()
        self.in_features, self.out_features = in_features, out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        # Fills the input `Tensor` with values according to the method described in
        # `Understanding the difficulty of training deep feedforward neural networks`
        # Glorot, X. & Bengio, Y. (2010)
        # using a uniform distribution.
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m, self.sin_m = math.cos(m), math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------
        one_hot = torch.zeros(cosine.size()).to('cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [ ]:
# https://github.com/haqishen/Google-Landmark-Recognition-2020-3rd-Place-Solution/blob/main/landmark-recognition-2020-third-place-submission.ipynb
import math

class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features*k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features
        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        
    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        
        return cosine 

In [ ]:
# src: https://amaarora.github.io/2020/08/30/gempool.html

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        # Applies 2D average-pooling operation in kH * kW regions by step size
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

# Creating the model

In [ ]:
def create_timm_body(arch:str, pretrained=True, drop_rate=0.0, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, drop_rate=drop_rate, 
                         num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")
    

In [ ]:
emb_size = 2048 # 2048 used b5_ns
body = create_timm_body('tf_efficientnet_b5_ns', pretrained=True)
nf = num_features_model(nn.Sequential(*body.children()))
head = nn.Sequential(GeM(),nn.Flatten(),
                     nn.Linear(nf,emb_size,bias=False))

model = nn.Sequential(body, head)

In [ ]:
loss_func = ArcMarginLoss(in_features=emb_size, out_features=dls.c,s=30.0,
                          m = 0.3, easy_margin=False, ls_eps=0.0)

In [ ]:
dls = create_dls(df=folds, bs=4,Image_size=512)
learn = Learner(dls, model, 
                loss_func=loss_func, 
                splitter=default_split
                ).to_fp16()
learn.freeze()

learn.summary()

In [ ]:
learn.lr_find() 

In [ ]:
learn.fit_one_cycle(1,lr_max=5e-4, wd = 1e-6)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(9,lr_max=5e-4)

In [ ]:
learn.export('effnet_b5_ns.pkl')

In [ ]:
del learn
torch.cuda.empty_cache()
gc.collect()

In [ ]:
def load_eval_learner(learner_path,output_group,dls,device):
    learn = load_learner(learner_path)
    learn.model.to(device)
    learn.dls = dls
    hook = Hook(learn.model[output_group], lambda m,i,o: o)
    return learn, hook

def load_dataloaders(train_df,test_df,val_fold):

    dls = create_dls(df=train_df, Image_size=512, bs=8, Val_Fold=val_fold) 

    train_dataloader = dls.test_dl(train_df[train_df.kfold != val_fold],with_labels=True)
    valid_dataloader = dls.test_dl(train_df[train_df.kfold == val_fold],with_labels=True)
    test_dataloader  = dls.test_dl(test_df)
    return train_dataloader, valid_dataloader, test_dataloader, dls

@torch.no_grad()
def get_embeddings(module, dataloader, device, train=True):

    all_image_names = []
    all_embeddings = []
    all_targets = []

    if train:
        
        for (x, y) in tqdm(dataloader):
            images =  x.to(device)
            targets = y.to(device)
            embeddings = module.model(images)
            all_embeddings.append(embeddings.cpu().numpy())
            all_targets.append(targets.cpu().numpy())


        all_image_names = dataloader.items['image_id'].values
        all_embeddings = np.vstack(all_embeddings)
        all_targets = np.concatenate(all_targets)
        all_targets = L(list(np.argmax(all_targets,axis=1)))
        all_embeddings = normalize(all_embeddings, axis=1, norm="l2")
        all_targets = np.array(all_targets.map(target_map.__getitem__))
      
        return all_image_names, all_embeddings, all_targets
    else:
        for (x,) in tqdm(dataloader):
            images =  x.to(device)
            embeddings = module.model(images)
            all_embeddings.append(embeddings.cpu().numpy())

        all_image_names = dataloader.items['image_id'].values
        all_embeddings = np.vstack(all_embeddings)
        all_embeddings = normalize(all_embeddings, axis=1, norm="l2")

        return all_image_names, all_embeddings


def create_and_search_index(embedding_size: int, train_embeddings: np.ndarray, val_embeddings: np.ndarray, k: int):
    index = faiss.IndexFlatIP(embedding_size)
    index.add(train_embeddings)
    D, I = index.search(val_embeddings, k=k)  

    return D, I

def create_val_targets_df(
    train_targets: np.ndarray, val_image_names: np.ndarray, val_targets: np.ndarray
) -> pd.DataFrame:

    allowed_targets = classes
    val_targets_df = pd.DataFrame(np.stack([val_image_names, val_targets], axis=1), columns=["image", "target"])
    val_targets_df.loc[~val_targets_df.target.isin(allowed_targets), "target"] = "new_turtle"

    return val_targets_df

def create_distances_df(
    image_names: np.ndarray, targets: np.ndarray, D: np.ndarray, I: np.ndarray, stage: str 
) -> pd.DataFrame:
    distances_df = []
    for i, image_name in tqdm(enumerate(image_names), desc=f"Creating {stage}_df"):
        target = targets[I[i]]
        distances = D[i]
        subset_preds = pd.DataFrame(np.stack([target, distances], axis=1), columns=["target", "distances"])
        subset_preds["image"] = image_name
        distances_df.append(subset_preds)
    distances_df = pd.concat(distances_df).reset_index(drop=True)
    distances_df = distances_df.groupby(["image", "target"]).distances.max().reset_index()
    distances_df = distances_df.sort_values("distances", ascending=False).reset_index(drop=True)
    return distances_df

def get_best_threshold(val_targets_df: pd.DataFrame, valid_df: pd.DataFrame):
    best_th = 0
    best_cv = 0
    for th in [0.1 * x for x in range(11)]:
        all_preds = get_predictions(valid_df, threshold=th)

        cv = 0
        for i, row in val_targets_df.iterrows():
            target = row.target
            preds = all_preds[row.image]
            val_targets_df.loc[i, th] = map_per_image(target, preds)

        cv = val_targets_df[th].mean()

        print(f"th={th} cv={cv}")

        if cv > best_cv:
            best_th = th
            best_cv = cv

    print(f"best_th={best_th}")
    print(f"best_cv={best_cv}")

    return best_th, best_cv

def get_predictions(df: pd.DataFrame, threshold: float = 0.2):

    predictions = {}
    for i, row in tqdm(df.iterrows(), total=len(df), desc=f"Creating predictions for threshold={threshold}"):
        if row.image in predictions:
            if len(predictions[row.image]) == 5:
                continue
            predictions[row.image].append(row.target)
        elif float(row.distances) > threshold:
            predictions[row.image] = [row.target, "new_turtle"]
        else:
            predictions[row.image] = ["new_turtle", row.target]

    for x in tqdm(predictions):
        if len(predictions[x]) < 5:
            predictions[x] = (predictions[x] + ['new_turtle']*5)[:5]

    return predictions

def map_per_image(label, predictions):
    """Computes the precision score of one image.

    Parameters
    ----------
    label : string
            The true label of the image
    predictions : list
            A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """
    try:
        return 1 / (predictions[:5].index(label) + 1)
    except ValueError:
        return 0.0


def create_predictions_df(test_df: pd.DataFrame, best_th: float) -> pd.DataFrame:
    predictions = get_predictions(test_df, best_th)

    predictions = pd.Series(predictions).reset_index()
    predictions.columns = ["image", "predictions"]
    predictions["predictions"] = predictions["predictions"].apply(lambda x: " ".join(x))

    return predictions

In [ ]:
def infer(
    model_path: str,
    train_df: pd.DataFrame = folds,
    test_df: pd.DataFrame = test_df,
    val_fold: float = 0,
    k: int = 50,
    emb_size:int = emb_size
):
    train_dl, val_dl, test_dl,dls = load_dataloaders(folds,test_df,val_fold)
    (learn, hook) = load_eval_learner(model_path,1,dls,torch.device("cuda"))

    train_image_names, train_embeddings, train_targets = get_embeddings(learn, train_dl,torch.device("cuda"))
    val_image_names, val_embeddings, val_targets = get_embeddings(learn, val_dl,torch.device("cuda"))
    test_image_names, test_embeddings = get_embeddings(learn, test_dl,torch.device("cuda"),train=False)

    D, I = create_and_search_index(emb_size, train_embeddings, val_embeddings, k) 
    print("Created index with train_embeddings")
    
    val_targets_df = create_val_targets_df(train_targets, val_image_names, val_targets)
    print(f"val_targets_df=\n{val_targets_df.head()}")
    print(f"val_targets_df shape=\n{val_targets_df.shape}")
    val_df = create_distances_df(val_image_names, train_targets, D, I, "val")
    print(f"val_df=\n{val_df.head()}")
    print(f"val_df shape=\n{val_df.shape}")
    best_th, best_cv = get_best_threshold(val_targets_df, val_df)
    print(f"val_targets_df=\n{val_targets_df.describe()}")
    print(f"val_targets_df shape=\n{val_targets_df.shape}")
    train_embeddings = np.concatenate([train_embeddings, val_embeddings])
    train_targets = np.concatenate([train_targets, val_targets])
    print("Updated train_embeddings and train_targets with val data")

    D, I = create_and_search_index(emb_size, train_embeddings, test_embeddings, k) 
    print("Created index with train_embeddings")

    test_df = create_distances_df(test_image_names, train_targets, D, I, "test")
    print(f"test_df=\n{test_df.head()}")
    print(f"test_df shape=\n{test_df.shape}")

    predictions = create_predictions_df(test_df, best_th)
    print(f"predictions.head()={predictions.head()}")
    print(f"predictions shape={predictions.shape}")

    predictions.to_csv(f'submission_{val_fold}.csv', index=False)

    print(f'Fold {val_fold} done!')

In [ ]:
for fold_num in range(5):
  
  infer(train_df = folds, test_df = test_df, model_path="effnet_b5_ns.pkl", val_fold=fold_num)

In [ ]:
sub_files = [
                 './submission_0.csv',
                 './submission_1.csv',
                 './submission_2.csv',
                 './submission_3.csv',
                 './submission_4.csv',

]

# Weights of the individual subs
# NB : Weights (best CV score per fold) are computed from get_best_threshold search
# Check best cv scores from the get_best_threshold outputs and insert for each placeholder below

sub_weight = [
                best_fold_1_cv_score**2,
                best_fold_2_cv_score**2,
                best_fold_3_cv_score**2,
                best_fold_4_cv_score**2,
                best_fold_5_cv_score**2
            ]

In [ ]:
Hlabel = 'image' 
Htarget = 'predictions'
npt = 5
place_weights = {}
for i in range(npt):
    place_weights[i] = ( 1 / (i + 1) )
    
print(place_weights)

lg = len(sub_files)
sub = [None]*lg
for i, file in enumerate( sub_files ):
   
    print("Reading {}: w={} - {}". format(i, sub_weight[i], file))
    reader = csv.DictReader(open(file,"r"))
    sub[i] = sorted(reader, key=lambda d: str(d[Hlabel]))

out = open("sub_ens.csv", "w", newline='')
writer = csv.writer(out)
writer.writerow([Hlabel,Htarget])

for p, row in enumerate(sub[0]):
    target_weight = {}
    for s in range(lg):
        row1 = sub[s][p]
        for ind, trgt in enumerate(row1[Htarget].split(' ')):
            target_weight[trgt] = target_weight.get(trgt,0) + (place_weights[ind]*sub_weight[s])
    tops_trgt = sorted(target_weight, key=target_weight.get, reverse=True)[:npt]
    writer.writerow([row1[Hlabel], " ".join(tops_trgt)])
out.close()

In [ ]:
result = pd.read_csv('/content/sub_ens.csv')
result

In [ ]:
pred_list = result.predictions.apply(lambda x : x.split(" ")).values.tolist()
result_df = pd.DataFrame(pred_list, columns=['prediction1', 'prediction2', 'prediction3', 'prediction4', 'prediction5'])
result_df['image_id'] = result['image'].copy()

result_df['image_id'] = result_df['image_id'].apply(lambda x : x.replace('.png', ''))
result_df = result_df[['image_id','prediction1', 'prediction2','prediction3','prediction4','prediction5']].copy()
result_df

In [ ]:
result_df.to_csv('./submission_b5_ns_fastai.csv.csv', index=False)